In [30]:
import psycopg2

# Function to read the db connection info
def read_db_connection_info(filename="../../db_connection_info.txt"):
    connection_info = {}
    with open(filename, 'r') as file:
        for line in file:
            key, value = line.strip().split('=')
            connection_info[key] = value
    return connection_info

# Load the connection info
connection_info = read_db_connection_info()

try:
    # Connect to the database
    conn = psycopg2.connect(
        host=connection_info["host"],
        user=connection_info["user"],
        password=connection_info["password"],
        dbname=connection_info["database"],
        port=connection_info["port"]
    )
    cursor = conn.cursor()

    
    # Query to find out how many ships each type of thing was used on
    query_ships_per_property = """
    SELECT thing, property, COUNT(DISTINCT ships_idx) AS ships_count
    FROM data_mapping
    WHERE ships_idx BETWEEN 1000 AND 1900
    AND thing <> 'ISS.AMS.DataGroup01.T' 
    AND thing <> 'ElectricEnergyConsumeThing' 
    AND thing <> '$UNMAPPED' 
    GROUP BY thing, property
    HAVING COUNT(DISTINCT ships_idx) >= 2
    ORDER BY ships_count DESC, thing, property;
    """
    cursor.execute(query_ships_per_property)  # Assuming query_ships_per_thing contains the updated SQL query

    # Fetch and print the result
    ships_per_property = cursor.fetchall()
    # Print the total number of properties
    print("Total number of properties:", len(ships_per_property))
    
    # Print details for each property
    print("Details of each property used on 2 or more ships:")
    for thing, property, ships_count in ships_per_property:
        print(f"Thing: {thing}, Property: {property}, Number of Ships: {ships_count}")

    

    # Close the cursor and connection
    cursor.close()
    conn.close()
    
except Exception as e:
    print(f"An error occurred: {e}")


Total number of properties: 545
Details of each property used on 2 or more ships:
Thing: DE1Thing, Property: FOInletTemp, Number of Ships: 50
Thing: DE2Thing, Property: FOInletTemp, Number of Ships: 50
Thing: DG1Thing, Property: WindingTempR, Number of Ships: 50
Thing: DG1Thing, Property: WindingTempS, Number of Ships: 50
Thing: DG1Thing, Property: WindingTempT, Number of Ships: 50
Thing: DG2Thing, Property: WindingTempR, Number of Ships: 50
Thing: DG2Thing, Property: WindingTempS, Number of Ships: 50
Thing: DG2Thing, Property: WindingTempT, Number of Ships: 50
Thing: DE1Thing, Property: CFWOutletTemp, Number of Ships: 49
Thing: DE1Thing, Property: Cy1ExhGasOutletTemp, Number of Ships: 49
Thing: DE1Thing, Property: Cy2ExhGasOutletTemp, Number of Ships: 49
Thing: DE1Thing, Property: Cy3ExhGasOutletTemp, Number of Ships: 49
Thing: DE1Thing, Property: Cy4ExhGasOutletTemp, Number of Ships: 49
Thing: DE1Thing, Property: Cy5ExhGasOutletTemp, Number of Ships: 49
Thing: DE1Thing, Property: FOI

In [31]:
import re

# Assuming ships_per_property is your list of tuples [(thing, property, ships_count), ...]

# Initialize an empty dictionary to hold the aggregated counts
aggregated_data = {}

for thing, property, ships_count in ships_per_property:
    # Replace all digits in the thing and property names with '#'
    modified_thing = re.sub(r'\d+', '#', thing)
    modified_property = re.sub(r'\d+', '#', property)

    # Initialize nested dictionary if the modified thing does not exist
    if modified_thing not in aggregated_data:
        aggregated_data[modified_thing] = {}

    # Aggregate ship counts by modified property under each modified thing
    if modified_property in aggregated_data[modified_thing]:
        aggregated_data[modified_thing][modified_property] += ships_count
    else:
        aggregated_data[modified_thing][modified_property] = ships_count

# Now, print the aggregated results for properties with ship_count > 1
for modified_thing in aggregated_data:    
    for modified_property, ships_count in aggregated_data[modified_thing].items():
        if ships_count >= 2:  # Check if ships_count is more than 1
            print(f"{modified_thing}, {modified_property}, {ships_count}")


DE#Thing, FOInletTemp, 159
DE#Thing, CFWOutletTemp, 157
DE#Thing, Cy#ExhGasOutletTemp, 1107
DE#Thing, FOInletPress, 155
DE#Thing, RPM, 155
DE#Thing, ScavTemp, 156
DE#Thing, ScavPress, 152
DE#Thing, CFWInletPress, 147
DE#Thing, CFWInletTemp, 144
DE#Thing, LOInletTemp, 138
DE#Thing, LOInletPress, 136
DE#Thing, DoUse, 63
DE#Thing, LTInletPress, 109
DE#Thing, LTInletTemp, 100
DE#Thing, FOFilterInletPress, 92
DE#Thing, LOFilterInletPress, 90
DE#Thing, LTOutletTemp, 82
DE#Thing, SAPress, 76
DE#Thing, LFOUse, 30
DE#Thing, FGMassFlow, 20
DE#Thing, FGPressure, 14
DE#Thing, FGTemperature, 14
DE#Thing, GasMode, 15
DE#Thing, HFOUse, 5
DE#Thing, LoadPercent, 12
DE#Thing, ULSFOUse, 4
DE#Thing, Load, 11
DE#Thing, RunningState, 8
DG#Thing, WindingTempR, 159
DG#Thing, WindingTempS, 159
DG#Thing, WindingTempT, 159
DG#Thing, Voltage, 144
DG#Thing, GenPower, 142
DG#Thing, Frequency, 140
DG#Thing, Current, 137
DG#Thing, RunningState, 136
DG#Thing, busBarVoltage, 40
DG#Thing, CBClosed, 38
DG#Thing, Load, 41

In [32]:
import csv

# Write the filtered aggregated results to a CSV file
with open('master_model.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    # Write the header
    csvwriter.writerow(['thing', 'property', 'ships_count'])

    # Write the data
    for modified_thing in aggregated_data:    
        for modified_property, ships_count in aggregated_data[modified_thing].items():
            csvwriter.writerow([modified_thing, modified_property, ships_count])
